In [1]:
import requests
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time 
import pandas as pd



Переходим по ссылке через selenium, открываем объявление, переходим во вкладку с ним

In [ ]:
url = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&object_type%5B0%5D=1&offer_type=flat&only_flat=1&region=1&room1=1&room2=1&room3=1"
browser = webdriver.Chrome()
page = browser.get(url)
advt = browser.find_elements(By.CLASS_NAME, '_93444fe79c--media--9P6wN')
advertisement = advt[0].click()
browser.switch_to.window(browser.window_handles[1])

Находим адрес, достаем оттуда, округ, район, улицу

In [5]:
address = [i.text for i in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--address-line--GRDTb')]
adm_distr = address[0].split(',')[1]
print(adm_distr)

 ЦАО


In [6]:
distr = address[0].split(',')[2].split(' ')[2]
print(distr)

Хамовники


In [7]:
st = address[0].split(',')[3]
print(st)

 ул. Ефремова


Из кратких характеристик достаем общую площадь,жилую площадь(если есть) площадь кухни(если есть), этаж, этажность дома

In [8]:
characteristics  = [j.text for j in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--Jp5Qv')]
characteristics


['Общая площадь\n78 м²', 'Площадь кухни\n10,2 м²', 'Этаж\n7 из 12']

In [9]:
total_ar = characteristics[0].split('\n')[1].split(' ')[0]
print(total_ar)

78


In [10]:
def find_living_area(characteristics):
    for k in characteristics:
        if k.split('\n')[0] == 'Жилая площадь':
            return k.split('\n')[1].split(' ')[0]
    return '-'
    
liv_area = find_living_area(characteristics)
print(liv_area)

-


In [11]:
def find_kitchen_area(characteristics):
    for k in characteristics:
        if k.split('\n')[0] == 'Площадь кухни':
            return k.split('\n')[1].split(' ')[0]
    return '-'

kit_area = find_kitchen_area(characteristics)
print(kit_area)

10,2


In [12]:
def find_flat_floor(characteristics):
    for k in characteristics:
        if k.split('\n')[0] == 'Этаж':
            return k.split('\n')[1].split(' ')[-3]
    return '-'

flat_fl = find_flat_floor(characteristics)
print(flat_fl)

7


In [13]:
def find_house_floors(characteristics):
    for k in characteristics:
        if k.split('\n')[0] == 'Этаж':
            return k.split('\n')[1].split(' ')[-1]
    return '-'
house_fl = find_house_floors(characteristics)
print(house_fl)

12


Если есть метро рядом, находим ближайшее и время до него

In [14]:
try:
    sub = [l.text for l in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--underground--pjGNr')]
    print(sub[0].split('\n')[0])
except:
    sub = '-'

Фрунзенская


In [15]:
if sub != '-':
    time_to_sub = sub[0].split('\n')[1]
    print(time_to_sub)
else: 
    time_to_sub = '-'

6 мин.


Находим количество комнат

In [16]:
preview = [m.text for m in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--container--u51hg')]
print(preview)

['Продается 2-комн. квартира, 78 м²\nв ЖК «Садовые кварталы»']


In [17]:
def number_of_rooms(preview):
    for i in preview:
        parts = i.split(' ')
        for j in parts:
            if '-' in j:
                room_info = j.split('-')
                if len(room_info) == 2 and room_info[1] == 'комн.':
                    return room_info[0]
    return '-'

room = number_of_rooms(preview)
print(room)

2


Находим цену, убираем пробелы и знак ₽

In [18]:
pr = [n.text for n in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--price--Pg6fn')]
pr = pr[0].replace('₽', '').replace(' ', '')
print(pr)

102999000


Во вторичной информации (при наличии) находим кол-во санузлов, тип ремонта, тип дома, год постройки, количество подъездов в доме, 

In [19]:
secondary_data = [i.text for i in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--group--K5ZqN')] 
print(secondary_data)

['О квартире\nТип жилья\nВторичка\nОбщая площадь\n78\nПлощадь кухни\n10,2\nСанузел\n2 совмещенных\nВид из окон\nВо двор\nРемонт\nЕвроремонт', 'О доме\nПодписаться на дом\nТип дома\nМонолитно-кирпичный\nПарковка\nМногоуровневая']


In [20]:
sublists = [j.split('\n') for j in secondary_data]
def number_of_bathrooms(sublists):
    for k in sublists:
        for ind, l in enumerate(k):
            if l == 'Санузел':
                return k[ind+1].split(' ')[0]
    return '-'

bathroom = number_of_bathrooms(sublists)
print(bathroom)

2


In [21]:
def renovation_type(sublists):
    for k in sublists:
        for ind, l in enumerate(k):
            if l == 'Ремонт':
                return k[ind+1]
    return '-'

renov = renovation_type(sublists)
print(renov)

Евроремонт


In [22]:
def housetype(sublists):
    for k in sublists:
        for ind, l in enumerate(k):
            if l == 'Тип дома':
                return k[ind+1]
    return '-'

house = housetype(sublists)
print(house)

Монолитно-кирпичный


In [23]:
def construct_year(sublists):
    for k in sublists:
        for ind, l in enumerate(k):
            if l == 'Год постройки':
                return k[ind+1]
    return '-'
year = construct_year(sublists)
print(year)

-


In [24]:
def num_of_entrances(sublists):
    for k in sublists:
        for ind, l in enumerate(k):
            if l == 'Подъезды':
                return k[ind+1]
    return '-'
entrances = num_of_entrances(sublists)
print(entrances)

-


In [25]:
adm_district = []
district = []
street = []
total_area = []
living_area = []
kitchen_area = []
flat_floor = []
house_floors = []
subway = []
time_to_subway = []
rooms = []
price = []
bathrooms = []
renovation = []
house_type = []
construction_year = []
number_of_entrances = [] 

функция для парсинга страницы Cian

In [173]:
def parcing(pages):
    browser = webdriver.Chrome()
    url = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&object_type%5B0%5D=1&offer_type=flat&only_flat=1&p=49&region=1&room1=1&room2=1&room3=1"
    page = browser.get(url)
    advt = browser.find_elements(By.CLASS_NAME, '_93444fe79c--media--9P6wN')
    for advtsmnt in advt:
        advtsmnt.click()
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(1)
        
        address = [i.text for i in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--address-line--GRDTb')]
        adm_distr = address[0].split(',')[1]
        adm_district.append(adm_distr)
        distr = address[0].split(',')[2].split(' ')[2]
        district.append(distr)
        st = address[0].split(',')[3]
        street.append(st)
        
        characteristics = [j.text for j in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--item--Jp5Qv')]
        
        total_ar = characteristics[0].split('\n')[1].split(' ')[0]
        total_area.append(total_ar)

        def find_living_area(characteristics):
            for k in characteristics:
                if k.split('\n')[0] == 'Жилая площадь':
                    return k.split('\n')[1].split(' ')[0]
            return '-'

        liv_area = find_living_area(characteristics)
        living_area.append(liv_area)

        def find_kitchen_area(characteristics):
            for k in characteristics:
                if k.split('\n')[0] == 'Площадь кухни':
                    return k.split('\n')[1].split(' ')[0]
            return '-'

        kit_area = find_kitchen_area(characteristics)
        kitchen_area.append(kit_area)

        def find_flat_floor(characteristics):
            for k in characteristics:
                if k.split('\n')[0] == 'Этаж':
                    return k.split('\n')[1].split(' ')[-3]
            return '-'

        flat_fl = find_flat_floor(characteristics)
        flat_floor.append(flat_fl)

        def find_house_floors(characteristics):
            for k in characteristics:
                if k.split('\n')[0] == 'Этаж':
                    return k.split('\n')[1].split(' ')[-1]
            return '-'

        house_fl = find_house_floors(characteristics)
        house_floors.append(house_fl)
        
        elements = browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--underground--pjGNr')
        if elements:
            sub = [l.text for l in elements]
            subway.append(sub[0].split('\n')[0])
            time_to_sub = sub[0].split('\n')[1]
            time_to_subway.append(time_to_sub)
        else:
            subway.append('-')
            time_to_subway.append('-')

        preview = [m.text for m in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--container--u51hg')]

        def number_of_rooms(preview):
            for i in preview:
                parts = i.split(' ')
                for j in parts:
                    if '-' in j:
                        room_info = j.split('-')
                        if len(room_info) == 2 and room_info[1] == 'комн.':
                            return room_info[0]
            return '-'

        room = number_of_rooms(preview)
        rooms.append(room)

        pr = [n.text for n in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--price--Pg6fn')]
        pr = pr[0].replace('₽', '').replace(' ', '')
        price.append(pr)
        
        secondary_data = [i.text for i in browser.find_elements(By.CLASS_NAME, 'a10a3f92e9--group--K5ZqN')]
        sublists = [j.split('\n') for j in secondary_data]

        def number_of_bathrooms(sublists):
            for k in sublists:
                for ind, l in enumerate(k):
                    if l == 'Санузел':
                        return k[ind+1].split(' ')[0]
            return '-'

        bathroom = number_of_bathrooms(sublists)
        bathrooms.append(bathroom)

        def renovation_type(sublists):
            for k in sublists:
                for ind, l in enumerate(k):
                    if l == 'Ремонт':
                        return k[ind+1]
            return '-'

        renov = renovation_type(sublists)
        renovation.append(renov)

        def housetype(sublists):
            for k in sublists:
                for ind, l in enumerate(k):
                    if l == 'Тип дома':
                        return k[ind+1]
            return '-'

        house = housetype(sublists)
        house_type.append(house)

        def construct_year(sublists):
            for k in sublists:
                for ind, l in enumerate(k):
                    if l == 'Год постройки':
                        return k[ind+1]
            return '-'

        year = construct_year(sublists)
        construction_year.append(year)

        def num_of_entrances(sublists):
            for k in sublists:
                for ind, l in enumerate(k):
                    if l == 'Подъезды':
                        return k[ind+1]
            return '-'

        entrances = num_of_entrances(sublists)
        number_of_entrances.append(entrances)

        browser.close()
        browser.switch_to.window(browser.window_handles[0])
    browser.close()
    

In [174]:
parcing(0)

Делаем из строковых данных численные

In [217]:
time_to_subway_ed = []
for i in time_to_subway:
    if i == '-':
        time_to_subway_ed.append(i) 
    elif  i ==  'откроется в 2024':
        time_to_subway_ed.append(i)
    else:
        i = i.split(' ')[0]
        i = int(i)
        time_to_subway_ed.append(i)

In [191]:
def str_to_int_ta(total_area):
    total_area_ed= []
    for i in total_area:
        i = i.replace(',', '.')
        i = float(i)
        total_area_ed.append(i)
    return total_area_ed
total_area = str_to_int_ta(total_area)
print((total_area))

In [202]:
def str_to_float_la(living_area):
    living_area_ed = []
    for i in living_area:
        if i == '-':
            living_area_ed.append(i)
        else:
            i = i.replace(',', '.')
            i = float(i)
            living_area_ed.append(i)
    return living_area_ed
living_area = str_to_float_la(living_area)


AttributeError: 'float' object has no attribute 'replace'

In [204]:
def str_to_float_k(kitchen_area):
    kitchen_area_ed = []
    for i in kitchen_area:
        if i == '-':
            kitchen_area_ed.append(i)
        else:
            i = i.replace(',', '.')
            i = float(i)
            kitchen_area_ed.append(i)
    return kitchen_area_ed
kitchen_area = str_to_float_k(kitchen_area)

AttributeError: 'float' object has no attribute 'replace'

In [205]:
rooms_ed = []
for i in rooms:
    i = int(i)
    rooms_ed.append(i)


In [211]:
price_ed = []
for i in price:
    if i == '':
        price_ed.append(i)
    else: 
        i = int(i)
        price_ed.append(i)

In [214]:
house_floors_ed = []
for i in house_floors:
    i = int(i)
    house_floors_ed.append(i)

In [219]:
flat_floor_ed = []
for i in flat_floor:
    i = int(i)
    flat_floor_ed.append(i)
print(flat_floor_ed)

[7, 5, 4, 5, 3, 8, 2, 3, 8, 7, 2, 12, 6, 10, 6, 3, 5, 4, 38, 3, 8, 8, 8, 10, 4, 8, 2, 8, 5, 8, 6, 2, 8, 3, 4, 2, 6, 4, 6, 3, 12, 6, 4, 25, 9, 4, 7, 5, 10, 10, 8, 34, 8, 6, 3, 3, 11, 17, 5, 4, 5, 4, 10, 5, 4, 4, 5, 11, 2, 9, 3, 1, 13, 40, 4, 9, 4, 4, 11, 8, 20, 3, 4, 3, 46, 3, 7, 8, 5, 3, 38, 4, 10, 8, 2, 3, 10, 7, 6, 2, 5, 7, 4, 2, 9, 2, 4, 37, 4, 2, 10, 3, 11, 5, 11, 1, 2, 5, 25, 6, 24, 46, 4, 2, 7, 10, 3, 3, 2, 5, 8, 4, 4, 38, 10, 2, 10, 5, 7, 10, 7, 3, 6, 4, 5, 2, 7, 7, 2, 2, 4, 9, 15, 2, 4, 10, 37, 4, 4, 2, 3, 2, 9, 3, 8, 3, 1, 5, 11, 11, 1, 13, 25, 9, 15, 2, 11, 18, 14, 2, 3, 6, 2, 3, 29, 26, 22, 9, 22, 3, 4, 9, 15, 14, 3, 4, 10, 5, 6, 3, 2, 11, 2, 5, 3, 3, 6, 2, 4, 8, 7, 9, 6, 15, 10, 4, 16, 6, 7, 5, 9, 6, 4, 18, 9, 4, 31, 4, 9, 8, 22, 4, 8, 4, 8, 7, 5, 5, 5, 6, 2, 9, 2, 9, 5, 1, 3, 6, 9, 5, 6, 21, 6, 13, 9, 8, 6, 3, 1, 10, 1, 2, 4, 1, 2, 12, 1, 29, 16, 1, 3, 45, 7, 2, 15, 21, 10, 28, 25, 4, 3, 3, 12, 2, 30, 3, 7, 2, 13, 2, 2, 8, 2, 2, 6, 3, 4, 4, 2, 5, 4, 6, 12, 1, 2, 5, 8, 10, 

In [221]:
bathrooms_ed = []
for i in bathrooms:
    if i == '-':
        bathrooms_ed.append(i)
    else: 
        i = int(i)
        bathrooms_ed.append(i)

In [225]:
number_of_entrances_ed = []
for i in number_of_entrances:
    if i == '-':
        number_of_entrances_ed.append(i)
    else: 
        i = int(i)
        number_of_entrances_ed.append(i)

Создаем дата фрейм

In [228]:
columns  = {'Район': district, 'Улица': street, 'Станция метро': subway, 'Время до метро, мин.': time_to_subway_ed, 'Количество комнат': rooms_ed, 'Общая площадь, м.кв.': total_area, 'Жилая площадь, м.кв.': living_area, 'Цена': price_ed, 'Год постройки': construction_year, 'Тип дома': house_type, 'Этаж': flat_floor_ed, 'Количество этажей в доме': house_floors_ed, 'Площадь кухни, м.кв.': kitchen_area, 'Количество санузлов': bathrooms, 'Тип ремонта': renovation, 'Количество подъездов в доме': number_of_entrances_ed} 

      

In [237]:
df = pd.DataFrame(columns)
df

,Район,Улица,Станция метро,"Время до метро, мин.",Количество комнат,"Общая площадь, м.кв.","Жилая площадь, м.кв.",Цена,Год постройки,Тип дома,Этаж,Количество этажей в доме,"Площадь кухни, м.кв.",Количество санузлов,Тип ремонта,Количество подъездов в доме
0,Хамовники,ул. Ефремова,Фрунзенская,6,2,78.00,-,102999000,-,Монолитно-кирпичный,7,12,10.2,2,Евроремонт,-
1,Пресненский,2-я Звенигородская ул.,Улица 1905 года,7,3,116.80,-,120000000,2023,-,5,21,-,2,Евроремонт,1
3,Пресненский,2-я Звенигородская ул.,Улица 1905 года,7,3,116.78,-,120000000,2023,-,5,21,-,2,Евроремонт,1
4,Пресненский,ул. Костикова,Улица 1905 года,9,2,51.00,-,52000000,2023,-,3,21,-,1,Евроремонт,-
5,Головинский,ул. Лавочкина,Водный стадион,4,2,44.00,27.0,10000000,1968,Панельный,8,9,7.0,1,Без ремонта,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,Лосиноостровский,Стартовая ул.,Медведково,3,3,65.60,40.7,17620000,1976,-,13,16,8.5,-,Косметический,1
1049,Донской,Варшавское ш.,Нагатинская,11,3,53.40,40.0,17550000,1962,Монолитный,8,8,5.3,1,Косметический,5
1050,Нижегородский,Рязанский просп.,Нижегородская,7,1,42.10,18.3,17700000,-,-,18,25,12.3,-,Евроремонт,-
1052,Покровское-Стрешнево,Волоколамское ш.,Трикотажная,5,1,39.10,20.0,17700000,-,-,21,29,12.0,1,Дизайнерский,-


Cохраняем в csv-файл без индексов

In [238]:
df.to_csv(r'cian.csv', index = False)
